In [1]:
import pickle
import os 
import cv2
import glob
import matplotlib.pyplot as plt

# select one of the multiple GPU cards on the server.
os.environ["CUDA_VISIBLE_DEVICES"]="0" # set GPU if multiple present




In [2]:
# the pkl directory contins the pkl files that hold a DeepLearning structure that defines the DL model
pklDir = input('Enter pkl path or use default\n') or '/home/dsi/aryeh/data/mia/pkl/'
# the model directory will receive the trained model
modelDir = input('Enter model output directory or default\n') or '/home/dsi/aryeh/data/mia/trained_models/'
    
loadweights = None

# modelname is the name of hte pkl file without its pkl  extension.
# a pkl file holds a data structure or object. IN this case, it is expected to hold a DeepLearning object that was saved
# using File>Save DL Object... in mianalyzer. 
modelname = input("Enter modelname without extension or use default") or "unetDensnet201_2023-01-24_08-17-37"


numclasses = 4

# trainingdata is a directory that holds the annotated images. The labels must be in trainingdata/Segmentation_labels/
# and they must be npz files.
trainingdata = '/home/dsi/aryeh/data/plants/unCropped/resized/'
validationdata = None

print("model: ", modelname)


model:  unetDensnet201_2023-01-24_08-17-37


In [3]:
print(os.getcwd())
os.chdir('/home/dsi/aryeh/data/mia/MIA/mia/')

/home/dsi/aryeh/data/mia/MIA/mia


In [4]:
        
print('load settings')
#    filehandler = open(modelname + '.pkl', 'rb')
filehandler = open(pklDir + modelname + '.pkl', 'rb')

print('opened file')

dl = pickle.load(filehandler)

# the dl parameters can be modified, either through methods in DeepLearning.py or by modifying public variables.
# here I can set teh number of epochs, so that a new pkl file does not need to be created when I change the number of epochs
dl.epochs=10

# it shold be possibleto define the entire structure wihtout a pkl file, but I have not yet figured out how to set or read 
# the architecture and backbone.

print('init model')
dl.initModel(numclasses)
print('load model')
if loadweights is not None:
    dl.Model.load_weights(loadweights)
        
if dl.initialized:
    print('start training')
    dl.initData_StartTraining(trainingdata, validationdata)
    print('training finished')
else:
    print('could not initialize model')
            
    
print('saving weights')
dl.Model.save_weights(modelDir+modelname + '.h5')
print('saving training data')
dl.saveTrainingRecord(modelDir+modelname +'.csv')
#   print('saving model')
#   dl.Model.save(modelDir+'full')
    



load settings
opened file


In [4]:
%matplotlib inline

In [ ]:
# This cell does a prediction on a directory of jpg images (could be any type, but that is what I have).
# predictionFolder contains the images which ill be predicted, and a subdirectory with eh modelname will be created
# to hold the predicted segmentation.

predictionFolder = input('Enter folder for prediction or use default\n')  or '/home/dsi/aryeh/data/plants/Harvest8Orange5_7Oct17/'
outputPath = predictionFolder + modelname+'_ep'+str(dl.epochs)+'/'
try:
    os.mkdir(outputPath)
except FileExistsError:
    pass
files = glob.glob(predictionFolder + '*.jpg')
print(len(files))

# If we do not want to process the entire folder, we can limit the number of processed image to count. 
count = 0
for i in files:
    print(i)
    img = cv2.imread(i)
    prediction = dl.Mode.PredictImage(img)
    plt.figure(count)
    plt.imshow(prediction)
    plt.show()
    cv2.imwrite(outputPath+'segmented_'+os.path.basename(i).replace('jpg', 'png'), prediction)
    count += 1
 #   if count > 1:
 #      break
    



In [1]:
print(dl.method.pixelbasedprediction.architecture)

NameError: name 'dl' is not defined

In [26]:
# I was checking why repeated runs of the prediction produced different reauslts. One possibility is that
# I need to reset the DL object between runs. But I am not sure about this.
print(dl.Model)
dl.cleanMemory()

0
